In [17]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

### 1. Вспомним распределение Бернулли и конверсию

Распределение Бернулли это распределение нулей и единиц, где 1 - это наступление какого-то события.

Это распределение характеризуется параметром 
$p$ - вероятностью наступления события


In [18]:
x= [1,1,1,0,0,0,0,0,0,0]

In [19]:
#считаем сумму наступления события 1, далее делим на кол-во всех событий
p = sum(x)/len(x)
print(p)

0.3


In [20]:
#cчитаем среднее распределения
np.mean(x)

0.3

In [21]:
#считаем дисперсию
p*(1-p)

0.21

Мы можем посчитать не только среднее (в случае с распределением Бернулли $p$), но и дисперсию по формуле: $Var = p*(1-p)$

Для сравнение двух выборок полученных из распределения Бернулли чаще всего применяют z-test для пропорций.

Шаг 1. Расчитываем оценку стандартной ошибки среднего:</br>
### $ESE = \sqrt{\frac{\bar{{p_1}}*(1-\bar{{p_1}})}{n_1}+\frac{\bar{{p_2}}*(1-\bar{{p_2}})}{n_2}}$

вместо стандартного отклонение по выборке в квадрате в формуле дисперсия расчитанная по выборке

Шаг 2. Расчитываем z-статистику по формуле:</br>
### $z = \frac{\bar{p_1} - \bar{p_2}}{ESE}$

### 2. z-test для пропорций и задача

**Задача**:</br>
Вы работаете в крупной продуктовой сети Покупочка. Маркетологи вашей компании решили поработать с  **оттоком**. В общем смысле отток - это ушедшие покупатели. 
А в вашей компании вы коллективно договорились считать оттоком покупателей, которые не приходили в магазин 2 месяца. При этом вы знаете, что чем раньше вы начинаете работать с отточным покупателем, тем выше вероятность его вернуть.

У маркетологов появилась идея: Мы будем отправлять отточным покупателям смс-ки с напоминанием о том, что у них есть накопленные бонусы. А они совершат покупку, чтобы списать бонусы.</br>

Пример смс: Илья, на вашем балансе 372 бонуса. Спишите их в Покупочке. 

Вы решили провести АБ тест. Отобрали покупателей, у которых последняя покупка была от 3 до 2 месяцев назад. Из них отобрали покупателей, у которых на балансе есть более 10 бонусов. Это будет ГС. 

Далее вы из ГС рандомно сформировали 2 выборки (тестовую и контрольную) по тысяче покупателей в каждой и отправили смс-ку тестовой выборке. 
Вы ожидаете, что в течении недели (период проведения АБ-теста) больше покупателей вернется из тестовой группы.





Период проведения АБ теста нужно выбирать кратным неделе, чтобы учесть недельную сезонность.

$H_0$ - Доли вернувшихся покупателей в группах не отличается </br>
$H_1$ - Доли вернувшихся покупателей в группах отличаются </br>
$\alpha$ = 0.05

In [22]:
p_test = 0.08  # Доля вернувшихся из тех, кто получил смс
p_control = 0.07  # Доля вернувшихся из тех, кто не получил смс

# Сгенерируем данные
np.random.seed(3)
test_group = np.random.choice([0, 1], size=1000, p=[1-p_test, p_test])
control_group = np.random.choice([0, 1], size=1000, p=[1-p_control, p_control])

In [23]:
np.mean(test_group)

0.08

In [24]:
np.mean(control_group)

0.072

In [25]:
z_stat, p_value = proportions_ztest([np.sum(test_group), np.sum(control_group)], [len(test_group), len(control_group)])
print("Z-statistic:", z_stat)
print("p-value:", p_value)

Z-statistic: 0.6750441957464585
p-value: 0.4996476862856807


p-value больше уровня значимости $\alpha$, значит у нас нет оснований отклонить нулевую гипотезу.</br>

Что если бы p-value было бы меньше 0.05? тогда бы мы отвергли нулевую гипотезу и сообщили бы маркетологам что тест успешен?

При принятии решения об успешности АБ теста, важно наблюдать не только за основной метрикой, но и за **guardrail** (охранными) метриками. Ваш АБ тест может повлиять и на другие важные метрики:
1. На сумму списанных бонусов
2. На сумму покупок (оборот)
3. На маржа
4. Маржа, очищенная от списанных бонусов

Давайте разбираться почему это важно. 

Представьте, что супермаркет Покупочка продает баночку колы. Цена баночки колы состоит из себестоймости и маржи.</br> **Себестоимость** - это цена закупки. Цена, которую мы заплатили поставщику. </br>
**Маржа** - это наценка магазина. 

Пример: кола стоит 100 рублей, себестоимость 80 рублей, а маржа 20 рублей. Если придет покупатель, купит колу, при этом 10 рублей оплатит бонусами, тогда этот минус ляжет в маржу. **Маржа, очищенная от списанных бонусов** составит 10 рублей.

Смс-ка с напоминанием о том, что у покупателя есть бонусы, скорее всего заставит часть покупателей прийти в магазин, но нужно еще учесть какая сумма бонусов спишется и какая будет маржа, очищенная от списанных бонусов.

Важно еще учитыватать, что одна смс стоит несколько рублей для компании. Также важно следить за тем, чтобы покупатель после покупки продолжил ходить в супермаркет.